In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('data/stud.csv')
df.head()

gender race_ethnicity parental_level_of_education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test_preparation_course  math_score  reading_score  writing_score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75

In [4]:
X = df.drop(columns=['math_score'], axis=1)
y = df['math_score']

In [5]:
X

gender race_ethnicity parental_level_of_education         lunch  \
0    female        group B           bachelor's degree      standard   
1    female        group C                some college      standard   
2    female        group B             master's degree      standard   
3      male        group A          associate's degree  free/reduced   
4      male        group C                some college      standard   
..      ...            ...                         ...           ...   
995  female        group E             master's degree      standard   
996    male        group C                 high school  free/reduced   
997  female        group C                 high school  free/reduced   
998  female        group D                some college      standard   
999  female        group D                some college  free/reduced   

    test_preparation_course  reading_score  writing_score  
0                      none             72             74  
1                 completed             90             88  
2                      none             95             93  
3                      none             57             44  
4                      none             78             75  
..                      ...            ...            ...  
995               completed             99             95  
996                    none             55             55  
997               completed             71             65  
998               completed             78             77  
999                    none             86             86  

[1000 rows x 7 columns]

In [6]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [7]:
X = preprocessor.fit_transform(X)

In [8]:
X.shape

(1000, 19)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [10]:
# Evaluation

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2

In [17]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)

    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    mae, mse, rmse, r2 = evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Performance Score for training")
    print(f"- MAE: {train_mae:.4f}")
    print(f"- MSE: {train_mse}:.4f")
    print(f"- RMSE: {train_rmse:.4f}")
    print(f"- r2: {train_r2:.4f}")

    print('----------------------------------------------------------')

    print("Performance Score for Test")
    print(f"- MAE: {mae:.4f}")
    print(f"- MSE: {mse:.4f}")
    print(f"- RMSE: {rmse:.4f}")
    print(f"- r2: {r2:.4f}")
    r2_list.append(r2)

    
    print('_'*70)

Linear Regression
Performance Score for training
- MAE: 4.2694
- MSE: 28.377889404296877:.4f
- RMSE: 5.3271
- r2: 0.8741
----------------------------------------------------------
Performance Score for Test
- MAE: 4.2195
- MSE: 29.3608
- RMSE: 5.4186
- r2: 0.8793
______________________________________________________________________
Lasso
Performance Score for training
- MAE: 5.2063
- MSE: 43.47840400585579:.4f
- RMSE: 6.5938
- r2: 0.8071
----------------------------------------------------------
Performance Score for Test
- MAE: 5.1579
- MSE: 42.5064
- RMSE: 6.5197
- r2: 0.8253
______________________________________________________________________
Ridge
Performance Score for training
- MAE: 4.2650
- MSE: 28.33778823308244:.4f
- RMSE: 5.3233
- r2: 0.8743
----------------------------------------------------------
Performance Score for Test
- MAE: 4.2111
- MSE: 29.0563
- RMSE: 5.3904
- r2: 0.8806
______________________________________________________________________
K-Neighbors Regressor

In [12]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [13]:
X.shape

(1000, 19)

In [18]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=['R2_Score'], ascending=False)

Model Name  R2_Score
2                    Ridge  0.880593
0        Linear Regression  0.879342
7       CatBoost Regressor  0.851632
5  Random Forest Regressor  0.851396
8       AdaBoost Regressor  0.851187
6             XGBRegressor  0.827797
1                    Lasso  0.825320
3    K-Neighbors Regressor  0.783813
4            Decision Tree  0.765779

In [19]:
ridge_regressor = Ridge(fit_intercept=True)
ridge_regressor = ridge_regressor.fit(X_train, y_train)
y_pred = ridge_regressor.predict(X_test)
score = r2_score(y_test, y_pred) * 100
print("Accuracy of the model %.2f" %score)

Accuracy of the model 88.06
